In [10]:
import random
import time

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import pandas as pd
import torch

from src.model.activation import Relu, LeakyRelu, Linear, Sigmoid
from src.model.base import NeuralNetwork
from src.model.layer import Convolutional, Input3D, Input, FullyConnected, Flatten
from src.model.loss import CategoricalCrossEntropy
from src.model.metric import AccuracyOneHot
from src.model.optimizer import Adam, SGD

In [11]:
# train_data = [
#     {
#         'input': [
#             [
#                 [0.0, 1.0, 1.0, 1.0, 0.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [0.0, 1.0, 1.0, 1.0, 0.0],
#             ]
#         ],
#         'output': [1.0, 0.0]
#     },
#     {
#         'input': [
#             [
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [0.0, 1.0, 0.0, 1.0, 0.0],
#                 [0.0, 0.0, 1.0, 0.0, 0.0],
#                 [0.0, 1.0, 0.0, 1.0, 0.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#             ]
#         ],
#         'output': [0.0, 1.0]
#     }    
# ]

# test_data = [
#     {
#         'input': [
#             [
#                 [1.0, 0.0, 1.0, 1.0, 0.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [0.0, 1.0, 1.0, 1.0, 0.0],
#             ]
#         ],
#         'output': [1.0, 0.0]
#     },
#     {
#         'input': [
#             [
#                 [0.0, 1.0, 0.0, 0.0, 1.0],
#                 [0.0, 1.0, 0.0, 1.0, 0.0],
#                 [0.0, 0.0, 1.0, 0.0, 0.0],
#                 [0.0, 1.0, 0.0, 1.0, 0.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#             ]
#         ],
#         'output': [0.0, 1.0]
#     }
    
# ]

In [12]:
# layers = [
#     Convolutional(LeakyRelu(), filters_num=16, kernel_size=2),
#     Convolutional(LeakyRelu(), filters_num=16, kernel_size=2),
#     Flatten(),
#     #FullyConnected(10, LeakyRelu()),
#     FullyConnected(2, Linear())
# ]
# nn = NeuralNetwork(
#     Input3D((1, 5, 5)),
#     layers,
#     optimizer=SGD(),
#     loss=CategoricalCrossEntropy(),
#     metric=AccuracyOneHot(),
#     convert_prediction='argmax'
# )


# start_time = time.time()
# epochs = 100
# nn.fit(train_data, test_data, epochs=epochs, verbose=True)
# print('exec time: ', time.time() - start_time)

# print('test_111')
# import sys
# sys.exit(0)

In [13]:
def get_onehot_digit(digit):
    output = [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    return output[-digit:] + output[:-digit]

In [14]:
def get_digits_data():

    train_dataset = []

    train = pd.read_csv("../data/digits/train.csv", header=0, nrows=5000)

    for index in train.index:
        input_values = [
            float(val) / 255.0 for val in train.loc[index].drop("label").values
        ]
        train_dataset.append(
            {
                "input": input_values,
                "output": get_onehot_digit(int(train.loc[index]["label"])),
            }
        )

    random.seed(1)
    random.shuffle(train_dataset)
    return train_dataset[:800], train_dataset[800:1000]

train_dataset, test_dataset = get_digits_data()

layers = [
    FullyConnected(256, LeakyRelu()),
    FullyConnected(128, LeakyRelu()),
    FullyConnected(64, LeakyRelu()),
    FullyConnected(10, Linear()),
]
nn = NeuralNetwork(
    Input(784),
    layers,
    optimizer=Adam(learning_rate=0.001),
    loss=CategoricalCrossEntropy(),
    metric=AccuracyOneHot(),
    convert_prediction="argmax",
    on_cuda=False
)

start_time = time.time()

epochs = 15
history = nn.fit(train_dataset, test_dataset, epochs=epochs, batch_size=1, verbose=True)

print("--- %s seconds ---" % (time.time() - start_time))

Epoch: 1/30, train loss: 1.9564, train Accuracy: 0.625, test loss: 1.8058, test Accuracy: 0.815, epoch time: 1.105s
Epoch: 2/30, train loss: 1.6986, train Accuracy: 0.8675, test loss: 1.6776, test Accuracy: 0.845, epoch time: 1.162s
Epoch: 3/30, train loss: 1.6143, train Accuracy: 0.9325, test loss: 1.7015, test Accuracy: 0.87, epoch time: 1.103s
Epoch: 4/30, train loss: 1.5668, train Accuracy: 0.9613, test loss: 1.6659, test Accuracy: 0.875, epoch time: 1.128s
Epoch: 5/30, train loss: 1.5308, train Accuracy: 0.9875, test loss: 1.6414, test Accuracy: 0.895, epoch time: 1.051s
Epoch: 6/30, train loss: 1.5088, train Accuracy: 0.99, test loss: 1.6321, test Accuracy: 0.875, epoch time: 1.107s
Epoch: 7/30, train loss: 1.4967, train Accuracy: 0.995, test loss: 1.6244, test Accuracy: 0.87, epoch time: 1.119s
Epoch: 8/30, train loss: 1.4983, train Accuracy: 0.9938, test loss: 1.6564, test Accuracy: 0.895, epoch time: 1.067s
Epoch: 9/30, train loss: 1.4932, train Accuracy: 0.9962, test loss: 1.

In [15]:
def plot_digit(image):
    image = torch.tensor(image).numpy().reshape(28, 28)
    plt.figure()
    plt.imshow(image, cmap="gray")
    plt.axis("off")
    plt.show()

    return

In [16]:
def get_digits_data_cnn():

    train_dataset = []

    train = pd.read_csv("../data/digits/train.csv", header=0, nrows=5000)

    for index in train.index:
        input_values = [
            float(val) / 255.0 for val in train.loc[index].drop("label").values
        ]
        train_dataset.append(
            {
                "input": [torch.tensor(input_values).reshape(28, 28).tolist()],
                "output": get_onehot_digit(int(train.loc[index]["label"])),
            }
        )

    # print(train_dataset[:2])
    # import sys
    # sys.exit(1)
    
    random.seed(1)
    random.shuffle(train_dataset)
    return train_dataset[:800], train_dataset[800:1000]

In [17]:
train_data, test_data = get_digits_data_cnn()

layers = [
    Convolutional(LeakyRelu(), filters_num=4, kernel_size=3),
    Convolutional(LeakyRelu(), filters_num=4, kernel_size=3),
    Convolutional(LeakyRelu(), filters_num=4, kernel_size=3),
    Flatten(),
    FullyConnected(128, LeakyRelu()),
    FullyConnected(10, Linear())
]
nn = NeuralNetwork(
    Input3D((1, 28, 28)),
    layers,
    optimizer=Adam(learning_rate=0.001),
    loss=CategoricalCrossEntropy(),
    metric=AccuracyOneHot(),
    convert_prediction='argmax',
    on_cuda=False
)

#plot_digit(train_data[0]['input'][0])

start_time = time.time()
epochs = 15
nn.fit(train_data, test_data, epochs=epochs, verbose=True)
print('exec time: ', time.time() - start_time)

print('test_111')

Epoch: 1/15, train loss: 1.939, train Accuracy: 0.7037, test loss: 1.7799, test Accuracy: 0.875, epoch time: 363.048s
Epoch: 2/15, train loss: 1.6976, train Accuracy: 0.9175, test loss: 1.7182, test Accuracy: 0.87, epoch time: 365.357s
Epoch: 3/15, train loss: 1.6057, train Accuracy: 0.9788, test loss: 1.6302, test Accuracy: 0.925, epoch time: 360.527s
Epoch: 4/15, train loss: 1.5695, train Accuracy: 0.9838, test loss: 1.7497, test Accuracy: 0.88, epoch time: 358.178s
Epoch: 5/15, train loss: 1.5402, train Accuracy: 0.9975, test loss: 1.6659, test Accuracy: 0.915, epoch time: 359.292s
Epoch: 6/15, train loss: 1.5138, train Accuracy: 1.0, test loss: 1.6235, test Accuracy: 0.925, epoch time: 357.846s
Epoch: 7/15, train loss: 1.5101, train Accuracy: 1.0, test loss: 1.6034, test Accuracy: 0.93, epoch time: 358.258s
Epoch: 8/15, train loss: 1.4963, train Accuracy: 1.0, test loss: 1.5893, test Accuracy: 0.92, epoch time: 359.548s
Epoch: 9/15, train loss: 1.4891, train Accuracy: 1.0, test los

In [18]:
# train_data = [
#     {
#         'input': [0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0],
#         'output': [1.0, 0.0]
#     },
#     {
#         'input': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0],
#         'output': [0.0, 1.0]
#     }    
# ]

# test_data = [
#     {
#         'input': [1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0],
#         'output': [1.0, 0.0]
#     },
#     {
#         'input': [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0],
#         'output': [0.0, 1.0]
#     }    
# ]

# layers = [
#     FullyConnected(25, LeakyRelu()),
#     FullyConnected(10, LeakyRelu()),
#     FullyConnected(2, Linear())
# ]
# nn = NeuralNetwork(
#     Input(25),
#     layers,
#     optimizer=SGD(),
#     loss=CategoricalCrossEntropy(),
#     metric=AccuracyOneHot(),
#     convert_prediction='argmax'
# )

# start_time = time.time()
# epochs = 200
# nn.fit(train_data, test_data, epochs=epochs, verbose=True)
# print('exec time: ', time.time() - start_time)

# print('test_222')